<a href="https://colab.research.google.com/github/anisaQode/hello-world/blob/master/Pollen_Traveller_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [3]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

Link to live google sheet

In [4]:
sheet1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1CxGecd0p2MaE9nr8aBDElibYuDo-aBo6QSoy_OBtUqo/edit?ts=6024605e#gid=1028397889')

In [5]:
ws = sheet1.worksheet('Campaigns')

In [6]:
Campaigns_df = get_as_dataframe(ws)

In [23]:
Campaigns_df.tail(5)

,Event Identifier,Region,Native Stage,Last Stage Change Date,Location: Name,Number of Nights,Start_Date,End_Date,On_sale_date_time,Event Type,Estimated_Travellers,Start_Month,Start_Week
1058,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1059,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1060,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1061,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1062,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


Drop unnecessary columns 


In [8]:
Campaigns_df.drop(['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'], axis=1, inplace=True)

View: Estimated travellers grouped by event type and location 

In [9]:
Campaigns_df.groupby(['Location: Name', 'Event Type'])['# Traveler Estimate'].sum()

Location: Name                  Event Type        
Amsterdam, Netherlands          ["Curated Travel"]    26334.0
                                ["Spring Break"]       8138.0
Barcelona, Spain                ["Curated Travel"]     2912.0
                                ["Spring Break"]       2118.0
Berlin, Germany                 ["Curated Travel"]     6625.0
Budapest, Hungary               ["Curated Travel"]     2488.0
Cabo San Lucas, Mexico          ["Curated Travel"]     2164.0
Cancun, Mexico                  ["Curated Travel"]    16738.0
Dubai, United Arab Emirates     ["Curated Travel"]     2969.0
Hawaii, United States           ["Curated Travel"]       20.0
Ibiza, Spain                    ["Curated Travel"]    21684.0
Lake Havasu, Arizona            ["Curated Travel"]     1590.0
Las Vegas, Nevada               ["Curated Travel"]    19328.0
                                ["Spring Break"]      10064.0
Les Deux Alpes, France          ["Curated Travel"]     1659.0
Malta              

View: Estimated travellers grouped by event type


In [10]:
Campaigns_travellers_by_event_type = Campaigns_df.groupby(['Event Type'])['# Traveler Estimate'].sum()

In [11]:
Campaigns_travellers_by_event_type

Event Type
["Curated Travel"]    131072.0
["Spring Break"]       20320.0
Name: # Traveler Estimate, dtype: float64

View: Estimated travellers grouped by location

In [12]:
Campaigns_location = Campaigns_df.groupby(['Location: Name'])['# Traveler Estimate'].sum().sort_values(ascending = False)

In [13]:
Campaigns_location

Location: Name
Amsterdam, Netherlands            34472.0
Las Vegas, Nevada                 29392.0
Ibiza, Spain                      21684.0
Cancun, Mexico                    16738.0
Berlin, Germany                    6625.0
San Juan, Puerto Rico              5779.0
Barcelona, Spain                   5030.0
Miami, Florida                     4551.0
Val Thorens, France                4302.0
Malta                              3574.0
Dubai, United Arab Emirates        2969.0
Palm Desert, California            2489.0
Budapest, Hungary                  2488.0
Cabo San Lucas, Mexico             2164.0
Punta Cana, Dominican Republic     2160.0
Tenerife, Spain                    1940.0
Mykonos, Greece                    1766.0
Les Deux Alpes, France             1659.0
Lake Havasu, Arizona               1590.0
Hawaii, United States                20.0
Name: # Traveler Estimate, dtype: float64

Estimated travellers grouped by region

In [14]:
Campaigns_region = Campaigns_df.groupby(['Region'])['# Traveler Estimate'].sum().sort_values(ascending = False)

In [15]:
Campaigns_region

Region
Europe           85970.0
North America    65422.0
Name: # Traveler Estimate, dtype: float64

Data clean


In [16]:
Campaigns_df.head(64)

,Event Identifier,Region,Native Stage,Last Stage Change Date,Location: Name,Number of Nights,Start Date,End Date,On Sale Date Time,Event Type,# Traveler Estimate
0,=3984258,North America,Prep,1/7/2021,"Punta Cana, Dominican Republic",4.0,5/20/2021,5/24/2021,12/15/2020,"[""Curated Travel""]",2160.0
1,=A2+175,Europe,Prep,1/12/2021,"Mykonos, Greece",4.0,9/24/2021,9/28/2021,NaN,"[""Curated Travel""]",735.0
2,=A3+175,North America,Prep,10/13/2020,"Las Vegas, Nevada",3.0,9/10/2021,9/13/2021,3/2/2021,"[""Spring Break""]",3605.0
3,=A4+175,Europe,Prep,10/27/2020,"Amsterdam, Netherlands",3.0,11/19/2021,11/22/2021,NaN,"[""Curated Travel""]",1563.0
4,=A5+175,Europe,Prep,2/1/2021,"Ibiza, Spain",4.0,9/8/2021,9/12/2021,NaN,"[""Curated Travel""]",4160.0
...,...,...,...,...,...,...,...,...,...,...,...
59,=A60+175,Europe,Cancelled,1/7/2021,"Dubai, United Arab Emirates",4.0,4/1/2021,4/5/2021,12/15/2020,"[""Curated Travel""]",909.0
60,=A61+175,Europe,Cancelled,6/30/2020,"Amsterdam, Netherlands",3.0,5/7/2021,5/10/2021,4/23/2020,"[""Curated Travel""]",1314.0
61,=A62+175,Europe,Cancelled,6/30/2020,"Val Thorens, France",6.0,3/6/2021,3/12/2021,5/21/2020,"[""Curated Travel""]",1221.0
62,=A63+175,Europe,Cancelled,7/27/2020,"Val Thorens, France",6.0,3/13/2021,3/19/2021,7/31/2020,"[""Curated Travel""]",3081.0


Data clean: Are there any rows that have all null values?

In [17]:
nan_df = Campaigns_df[Campaigns_df.isna().any(axis=1)]
nan_df.head()

,Event Identifier,Region,Native Stage,Last Stage Change Date,Location: Name,Number of Nights,Start Date,End Date,On Sale Date Time,Event Type,# Traveler Estimate
1,=A2+175,Europe,Prep,1/12/2021,"Mykonos, Greece",4.0,9/24/2021,9/28/2021,NaN,"[""Curated Travel""]",735.0
3,=A4+175,Europe,Prep,10/27/2020,"Amsterdam, Netherlands",3.0,11/19/2021,11/22/2021,NaN,"[""Curated Travel""]",1563.0
4,=A5+175,Europe,Prep,2/1/2021,"Ibiza, Spain",4.0,9/8/2021,9/12/2021,NaN,"[""Curated Travel""]",4160.0
5,=A6+175,Europe,Prep,NaN,"Barcelona, Spain",3.0,9/3/2021,9/6/2021,NaN,"[""Spring Break""]",2118.0
6,=A7+175,North America,Prep,NaN,"Las Vegas, Nevada",4.0,9/23/2021,9/27/2021,NaN,"[""Curated Travel""]",3175.0


In [18]:
# Augment data with additional columns , Add month columns

Add month and week column, rename columns for easier access

In [19]:
#Campaigns_df['Month'] = Campaigns_df['Start Date'].str[0:2]
#Campaigns_df['Month'] = Campaigns_df['Month'].astype('int32')
#Campaigns_df.head()
Campaigns_df.rename(columns={'Start Date':'Start_Date', 'End Date':'End_Date', '# Traveler Estimate':'Estimated_Travellers','On Sale Date Time':'On_sale_date_time' }, inplace=True)
Campaigns_df['Start_Date'] = pd.to_datetime(Campaigns_df.Start_Date)
Campaigns_df['Start_Month'] = Campaigns_df.Start_Date.dt.month
Campaigns_df['Start_Week'] = Campaigns_df.Start_Date.dt.week

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  import sys


Stage to status mapping


In [20]:
Campaigns_df['Native Stage'] = Campaigns_df['Native Stage'].replace({'Prep': 'Pre-Sale', 'Campaign Setup Required': 'Pre-Sale', 'Hype': 'Pre-Sale', 'Quality Assurance Review': 'Pre-Sale', 'Pre-Event Ops Planning':'Active', 'On-Sale' : 'Active', 'Postponed':'Postponed', 'Cancelled':'Cancelled'})

What trip status are most estimated travellers on?

In [21]:
Campaigns_stage = Campaigns_df.groupby(['Native Stage'])['Estimated_Travellers'].sum()
Campaigns_stage

Native Stage
Active       89655.0
Cancelled     6545.0
Postponed    15687.0
Pre-Sale     39505.0
Name: Estimated_Travellers, dtype: float64

In [22]:
Stages = [stage for stage, df in Campaigns_df.groupby(['Native Stage'])]
plt.scatter(Stages, Campaigns_stage)
plt.xticks(Stages)
plt.ylabel('Estimated Travellers')
plt.xlabel('Stages (2021)')

NameError: ignored

Monthly view: What month may we see the most travellers? How many travellers can we expect?


In [ ]:
monthly_travellers = Campaigns_df.groupby('Start_Month').sum()['Estimated_Travellers']
results.head(13)

In [ ]:
import matplotlib.pyplot as plt
months = range(3, 13)
plt.plot(months, monthly_travellers)
plt.xticks(months)
plt.ylabel('Estimated Travellers')
plt.xlabel('Months (2021)')

Weekly view: Estimated travellers per week

In [ ]:
weekly_travellers = Campaigns_df.groupby('Start_Week').sum()['Estimated_Travellers']
weekly_travellers.head(52)

In [ ]:
weeks = [week for week, df in Campaigns_df.groupby(['Start_Week'])]
plt.plot(weeks, weekly_travellers)
plt.xticks(weeks, rotation = 'vertical', size = 7)
plt.ylabel('Estimated Travellers')
plt.xlabel('Weeks (2021)')


How many travellers are on Spring Break vs Curated Travel trips?
What Location has the higest number of estimated travellers?

In [ ]:
Campaigns_travellers_by_event_type = Campaigns_df.groupby(['Event Type'])['Estimated_Travellers'].sum()
Campaigns_travellers_by_event_type.head()

In [ ]:
Estimated_Travellers = ['131052', '20320']
Event_Type_Labels = ['Curated Travel', 'Spring Break']
plt.pie(Estimated_Travellers, labels = Event_Type_Labels, radius = 1.5, autopct = '%0.1f%%', shadow = True, explode = [0, 0.1])
plt.show()

In [ ]:
Campaigns_location = Campaigns_df.groupby(['Location: Name'])['Estimated_Travellers'].sum().sort_values()
Campaigns_location

In [ ]:
Cities = [city for city, df in Campaigns_df.groupby(['Location: Name'])]
plt.bar(Cities, Campaigns_location)
plt.xticks(Locations, rotation = 'vertical', size = 8)
plt.ylabel('Estimated Travellers')
plt.xlabel('Locations (2021)')

~When should we put an event on-sale to maximise likelihood of customers buying a ticket/package?

In [ ]:
Campaigns_df.columns

In [ ]:
Campaigns_df.dtypes

In [ ]:
Campaigns_df['On_sale_date_time'] = pd.to_datetime(Campaigns_df['On_sale_date_time'])

In [ ]:
#Need timestamp and booking table to figure out best time to put up an event (using list comprehension to go through hours and grouping by each hour, aggregating with count, plotting a line graph)